In [1]:
import os 

for dirname, _, filenames in os.walk('./input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import numpy as np
import pandas as pd

./input/titanic.zip
./input/gender_submission.csv
./input/test.csv
./input/train.csv
./input/titanic.zip:Zone.Identifier


In [2]:
train =pd.read_csv('./input/train.csv')
test = pd.read_csv('./input/test.csv')
gender_submission = pd.read_csv('./input/gender_submission.csv')
data = pd.concat([train,test],sort = False)
data['Sex']=data['Sex'].replace(['male','female'],[0,1])
data['Fare']=data['Fare'].fillna(np.mean(data['Fare']))
data['Embarked']= data['Embarked'].fillna('S').map({'S':0,'C':1,'Q':2}).astype(int)
age_ave = data['Age'].mean()
age_std = data['Age'].std()
data['Age'] = data['Age'].fillna(np.random.randint(age_ave - age_std,age_ave+ age_std))
delete_columns = ['Name', 'PassengerId', 'Parch', 'Ticket', 'Cabin']
data = data.drop(delete_columns, axis=1)

/tmp/ipykernel_9751/3889284680.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['Sex']=data['Sex'].replace(['male','female'],[0,1])


In [3]:
train = data[:len(train)]
test = data[len(train):]
#行数で分離

In [4]:
y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)
#最終結果だけ分離

In [5]:
from sklearn.model_selection import train_test_split


X_train, X_valid, y_train, y_valid = \
    train_test_split(X_train, y_train, test_size=0.3,
                                 random_state=0, stratify=y_train)


In [6]:
categorical_features = ['Embarked', 'Pclass', 'Sex']
#カテゴリ変数だと宣言しないと連続数とみなされる。

In [13]:
import lightgbm as lgb
import optuna
from sklearn.metrics import log_loss
from lightgbm import early_stopping, log_evaluation

In [18]:
def objective(trial):
    params = {
        'objective': 'binary',
        'max_bin': trial.suggest_int('max_bin', 255, 500),
        'learning_rate': 0.05,
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }

    lgb_train = lgb.Dataset(X_train, y_train,
                                             categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train,
                                            categorical_feature=categorical_features)
    callbacks = [lgb.early_stopping(stopping_rounds=10)]

    model = lgb.train(params, lgb_train,
                                   valid_sets=[lgb_train, lgb_eval],
                                   num_boost_round=1000,
                                   callbacks=callbacks)

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    score = log_loss(y_valid, y_pred_valid)
    return score

In [ ]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)


[I 2025-12-27 21:59:26,697] A new study created in memory with name: no-name-3e675e36-0f83-41d0-b3b9-231b246dd838


[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.030694 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 188
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383628 -> initscore=-0.474179
[LightGBM] [Info] Start training from score -0.474179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

[I 2025-12-27 21:59:27,947] Trial 0 finished with value: 0.43649371933258874 and parameters: {'max_bin': 390, 'num_leaves': 101}. Best is trial 0 with value: 0.43649371933258874.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[45]	training's binary_logloss: 0.329344	valid_1's binary_logloss: 0.436494
[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003207 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 188
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383628 -> initscore=-0.474179
[LightGBM] [Info] Start training from score -0.474179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive 

[I 2025-12-27 21:59:30,174] Trial 1 finished with value: 0.43649371933258874 and parameters: {'max_bin': 403, 'num_leaves': 84}. Best is trial 0 with value: 0.43649371933258874.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[45]	training's binary_logloss: 0.329344	valid_1's binary_logloss: 0.436494
[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003050 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 188
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383628 -> initscore=-0.474179
[LightGBM] [Info] Start training from score -0.474179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't 

[I 2025-12-27 22:00:48,884] Trial 38 finished with value: 0.43649371933258874 and parameters: {'max_bin': 324, 'num_leaves': 43}. Best is trial 0 with value: 0.43649371933258874.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[45]	training's binary_logloss: 0.329344	valid_1's binary_logloss: 0.436494
[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002923 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 188
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383628 -> initscore=-0.474179
[LightGBM] [Info] 

[I 2025-12-27 22:00:51,767] Trial 39 finished with value: 0.43649371933258874 and parameters: {'max_bin': 327, 'num_leaves': 43}. Best is trial 0 with value: 0.43649371933258874.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[45]	training's binary_logloss: 0.329344	valid_1's binary_logloss: 0.436494


In [20]:
study.best_params

{'max_bin': 390, 'num_leaves': 101}

In [ ]:




lgb_train = lgb.Dataset(X_train, y_train,
                                         categorical_feature=categorical_features)
#reference= はlgb_trainと同じ前処理を適用するため
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train,
                                         categorical_feature=categorical_features)
#params(条件設定)、objective(目的関数)を'binary'（二値分類）に設定ほかにregressionなどもある
#max_bin(特徴量のビンの最大数,分割数)、learning_rate(学習率)通常0.1、num_leaves(決定木の葉の数)通常31
params = {
    'objective': 'binary',
    'max_bin': study.best_params['max_bin'],
    'learning_rate': 0.05,
    'num_leaves': study.best_params['num_leaves']
}
# ログ出力と早期終了のための機能をインポート
from lightgbm import log_evaluation, early_stopping

# コールバック関数をリストにまとめる
#10回ごとにログを出力、10回連続で改善しなければ早期終了
callbacks = [
        # verbose_eval=10 の代わり
    early_stopping(stopping_rounds=10) # early_stopping_rounds=10 の代わり
]

# 学習の実行,valid_setsで評価データを指定,num_boost_roundで最大繰り返し回数を指定
model = lgb.train(
    params, 
    lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    num_boost_round=1000,
    callbacks=callbacks 
)
#num_iteration=model.best_iterationで一番精度の良い回数を指定
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004326 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 188
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383628 -> initscore=-0.474179
[LightGBM] [Info] Start training from score -0.474179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Wa

In [22]:
y_pred[:10]

array([0.08749611, 0.54470333, 0.14983269, 0.07892528, 0.46530227,
       0.43738766, 0.68895359, 0.12264774, 0.72326217, 0.04796149])

In [23]:
y_pred = (y_pred > 0.5).astype(int)
y_pred[:10]

array([0, 1, 0, 0, 0, 0, 1, 0, 1, 0])

In [ ]:
sub = pd.read_csv('./input/gender_submission.csv')
sub['Survived'] = y_pred
sub.to_csv('submission_lightgbm-optuna1.csv', index=False)

sub.head()
#kaggleでのスコアは0.76794で変わらず

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0


In [26]:
import optuna.visualization as vis

In [27]:
vis.plot_optimization_history(study)

In [28]:
vis.plot_param_importances(study)

In [29]:
vis.plot_parallel_coordinate(study)

In [30]:
print(study.trials_dataframe())

    number     value             datetime_start          datetime_complete  \
0        0  0.436494 2025-12-27 21:59:26.699309 2025-12-27 21:59:27.947428   
1        1  0.436494 2025-12-27 21:59:27.948857 2025-12-27 21:59:30.174293   
2        2  0.436494 2025-12-27 21:59:30.175884 2025-12-27 21:59:32.657775   
3        3  0.436494 2025-12-27 21:59:32.660457 2025-12-27 21:59:34.463735   
4        4  0.436494 2025-12-27 21:59:34.466332 2025-12-27 21:59:36.300200   
5        5  0.436494 2025-12-27 21:59:36.301701 2025-12-27 21:59:38.187083   
6        6  0.436494 2025-12-27 21:59:38.189700 2025-12-27 21:59:40.042606   
7        7  0.436494 2025-12-27 21:59:40.044112 2025-12-27 21:59:41.946384   
8        8  0.436494 2025-12-27 21:59:41.947339 2025-12-27 21:59:44.209071   
9        9  0.436494 2025-12-27 21:59:44.210563 2025-12-27 21:59:46.269433   
10      10  0.436494 2025-12-27 21:59:46.270963 2025-12-27 21:59:48.114270   
11      11  0.436494 2025-12-27 21:59:48.115737 2025-12-27 21:59